# Packing Density & Mixing Index Demo (Julia)

This notebook demonstrates how to use the `Packing3D` Julia package to:

1. Load a sample particle dataset  
2. Compute packing density  
3. Compute mixing indices  

The first code cell below is a simple example of using Packing3D to calculate the packing density of particles in a cylindrical vessel. Feel free to modify the boundaries by a few millimetres to see how this affects the packing density.

In [ ]:
# Packing Density in a Cylindrical Vessel #

using Packing3D

file = "particles_0.vtk"

data = read_vtk_file(file)

boundaries_cylindrical = Dict(
    :r_min => -1, :r_max => 0.025,
    :theta_min => 0.0, :theta_max => 2*pi,
    :z_min => 0.005, :z_max => 0.03
)

packing_density = calculate_packing(;
    data=data,
    boundaries=boundaries_cylindrical,
    system=:cylindrical
)

println("Packing Density: ", packing_density)

The next block demonstrates a comparison between the first and last file in from a vibro-packing simulation:

In [ ]:
# Packing density change after vibro-packing #

using Packing3D

# Load files
file_initial = "particles_0.vtk"
file_final   = "particles_final.vtk"

data_initial = read_vtk_file(file_initial)
data_final   = read_vtk_file(file_final)

boundaries_cylindrical = Dict(
    :r_min => -1, :r_max => 0.025,
    :theta_min => 0.0, :theta_max => 2*pi,
    :z_min => 0.005, :z_max => 0.03
)

packing_initial = calculate_packing(;
    data=data_initial,
    boundaries=boundaries_cylindrical,
    system=:cylindrical
)

packing_final = calculate_packing(;
    data=data_final,
    boundaries=boundaries_cylindrical,
    system=:cylindrical
)

println("Initial packing density:", packing_initial)
println("Final packing density  :", packing_final)

If you run the above, you will see a modest increase in packing density in this simulation. Let's see how well the bed stays mixed...

In [ ]:
# Lacey mixing index change after vibro-packing #

using Packing3D

# Load files
file_initial = "particles_0.vtk"
file_final   = "particles_final.vtk"

data_initial = read_vtk_file(file_initial)
data_final   = read_vtk_file(file_final)

# Gather particle ids of the small and large particles from the initial file
data_1_ids, data_2_ids = split_data(
    data_initial;
    split_by=:radius,
    threshold=0.0007
)

# Match these ids to the relevant particles in each dataset
data_1_initial, data_2_initial = match_split_data(
    data_initial, data_1_ids, data_2_ids
)
data_1_final, data_2_final = match_split_data(
    data_final, data_1_ids, data_2_ids
)

# Define cylinder parameters
cylinder_params = Dict(
    :cylinder_radius => 0.03,
    :cylinder_base_level => 0.0,
    :cylinder_height => 0.08
)

# Construct a cylindrical Mesh with 1000 cells
cylindrical_mesh = Mesh(
    :cylindrical;
    params=cylinder_params,
    target_num_cells=1000
)

lacey_initial = calculate_lacey(
    data_1_initial, data_2_initial;
    mesh=cylindrical_mesh,
    calculate_partial_volumes=true
)

lacey_final = calculate_lacey(
    data_1_final, data_2_final;
    mesh=cylindrical_mesh,
    calculate_partial_volumes=true
)

println("Initial Lacey mixing index:", lacey_initial)
println("Final Lacey mixing index  :", lacey_final)

If you run the above, you will unfortunately see a clear segregative result indicated by the drop in Lacey Index.